# **2752. Customers with Maximum Number of Transactions on Consecutive Days**

**Table: Transactions**
``` sql
+------------------+------+
| Column Name      | Type |
+------------------+------+
| transaction_id   | int  |
| customer_id      | int  |
| transaction_date | date |
| amount           | int  |
+------------------+------+
``` 
transaction_id is the column with unique values of this table.

Each row contains information about transactions that includes unique (customer_id, transaction_date) along with the corresponding customer_id and amount.   

Write a solution to find all customer_id who made the maximum number of transactions on consecutive days.

Return all customer_id with the maximum number of consecutive transactions. Order the result table by customer_id in ascending order.

The result format is in the following example.

 

**Example 1:**

**Input:** 
**Transactions table:**
``` sql
+----------------+-------------+------------------+--------+
| transaction_id | customer_id | transaction_date | amount |
+----------------+-------------+------------------+--------+
| 1              | 101         | 2023-05-01       | 100    |
| 2              | 101         | 2023-05-02       | 150    |
| 3              | 101         | 2023-05-03       | 200    |
| 4              | 102         | 2023-05-01       | 50     |
| 5              | 102         | 2023-05-03       | 100    |
| 6              | 102         | 2023-05-04       | 200    |
| 7              | 105         | 2023-05-01       | 100    |
| 8              | 105         | 2023-05-02       | 150    |
| 9              | 105         | 2023-05-03       | 200    |
+----------------+-------------+------------------+--------+
```
**Output:** 
``` sql
+-------------+
| customer_id | 
+-------------+
| 101         | 
| 105         | 
+-------------+
```
**Explanation:** 
- customer_id 101 has a total of 3 transactions, and all of them are consecutive.
- customer_id 102 has a total of 3 transactions, but only 2 of them are consecutive. 
- customer_id 105 has a total of 3 transactions, and all of them are consecutive.

In total, the highest number of consecutive transactions is 3, achieved by customer_id 101 and 105. The customer_id are sorted in ascending order.

**Solution-1:**
``` sql
WITH rn_info AS (
SELECT ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY transaction_date) AS rn,
customer_id, transaction_date 
FROM (SELECT DISTINCT customer_id, transaction_date FROM Transactions) temp),

customer_with_base_date AS (
SELECT customer_id, DATE_SUB(transaction_date, INTERVAL rn DAY) AS base_date FROM rn_info),

customer_with_count AS (
SELECT customer_id,
COUNT(*) AS c 
FROM customer_with_base_date 
GROUP BY customer_id, base_date)

SELECT customer_id FROM customer_with_count 
WHERE c IN (SELECT MAX(c) FROM customer_with_count)

**Solution-2:**
``` sql
cte1 as
(
    select customer_id, 
    (datediff(transaction_date, min(transaction_date) over(partition by customer_id))) + 1 - (row_number() over(partition by customer_id order by transaction_date)) diff
    from transactions
)
,
cte2 as
(
    select customer_id, count(*) cnt
    from cte1
    group by customer_id, diff
)

select customer_id
from cte2
where cnt = (select max(cnt) from cte2) 
order by customer_id